In [ ]:
%run 'common-functions.ipynb'

In [ ]:
CLOUDTRAIL_STARTSWITH_FILTER = "D"

In [ ]:
@dask.delayed
def get_cloudtrails(client, account_id, account_region):
    paginator = client.get_paginator("list_trails")
    page_iterator = paginator.paginate()
    data = []
    data = [page["Trails"] for page in page_iterator]
    df = pd.DataFrame([item for sublist in data for item in sublist])
    df["account_id"] = account_id
    df["region"] = account_region
    return df


In [ ]:
accounts = fetch_accounts_from_metadata()


In [ ]:
%%time
# Fetch all cloudwatch log groups across all spokes that dont have a retentionInDays value set
master_df = pd.DataFrame()
dfs = []
print(f"Accounts to process: {len(accounts)}")
for account in accounts:
    account_id = account.get("account")
    account_region = account.get("region")
    if account_region:
        client = assume_return_boto_client("cloudtrail", account_id, account_region)
        dfs.append(get_cloudtrails(client, account_id, account_region))

graph = dask.delayed()(dfs)
results = graph.compute()
master_df = pd.concat(results)

In [ ]:
master_df


In [ ]:
filtered = master_df[master_df["Name"].str.startswith(CLOUDTRAIL_STARTSWITH_FILTER)]
filtered.to_csv("./cloudtrails-filtered.csv")
